# Object Detection
> 객체 탐지 논문

- toc: true 
- badges: true
- comments: true
- categories: [python]
- image: images/

---


참고 사이트 : https://ganghee-lee.tistory.com/

# 1. R-CNN (2013)
***Rich feature hierarchies for accurate object detection and semantic segmentation***

원문 : https://arxiv.org/abs/1311.2524

<img src='img/Detection/rcnn.png' width="500" height="500">

" R-CNN 프로세스 "

1. Image를 입력받는다.

2. Selective search 알고리즘에 의해 regional proposal output 약 2000개를 추출한다.

    (추출한 regional proposal output을 모두 동일 input size로 만들어주기 위해 warp 해준다.
    
    `?` 왜 동일 input size로 만들어 줄까? : 사실 Convolution Layer에는 input size가 고정이지 않다. 그러나 마지막 FC layer에서의 input size는 고정이므로 Convolution Layer에 입력에서부터 동일한 input size로 넣어주어 output size를 동일하게 하는 것)

    
3. 2000개의 warped image를 각각 CNN 모델에 넣는다.

4. 각각의 Convolution 결과에 대해 classification을 진행하여 결과를 얻는다.



위의 과정을 수행하기 위해 R-CNN은 세 가지 모듈로 나누어 놓았다.

- 1. Region Proposal : "Object가 있을법한 영역"을 찾는 모듈 (기존의 Sliding window방식의 비효율성 극복)
    
- 2. CNN : 각각의 영역으로부터 고정된 크기의 Feature Vector를 뽑아낸다.

- 3. SVM : Classification을 위한 선형 지도학습 모델

    `?` 왜 Classifier로 Softmax를 쓰지 않고 SVM을 사용했을까? : CNN fine-tuning을 위한 학습 데이터가 시기상 많지 않아서 Softmax를 적용시키면 오히려 성능이 낮아져 SVM을 사용

## 1. Region Proposal

<img src='img/Detection/region.png' width="500" height="500">

R-CNN에서는 가장 먼저 Region Proposal 단계에서 "물체가 있을 법한 영역"을 찾는다.

이는 위에서 말했듯이 기존의 Sliding window방식의 비효율성을 극복하기 위한 것이다.

먼저 기존의 Sliding window가 무엇인지 살펴보자.

### Sliding Window

Sliding window방식은 이미지에서 물체를 찾기 위해 window의 (크기, 비율)을 임의로 마구 바꿔가면서
모든 영역에 대해서 탐색하는 것이다.

<img src='img/Detection/sliding.png' width="500" height="500">

<center>좌 : 모든 영역에 대해 탐색 / 우 : 크기와 비율을 변형</center>

이렇게 임의의 (크기, 비율)로 모든 영역을 탐색하는 것은 너무 느리다.

따라서 R-CNN에서는 이 비효율성을 극복하기 위해 Selective search 알고리즘을 사용한다.

### Selective search

<img src='img/Detection/selective.png' width="500" height="500">

1. 색상, 질감, 영역크기 등.. 을 이용해 non-object-based segmentation을 수행한다.

    이 작업을 통해 좌측 제일 하단 그림과 같이 많은 small segmented areas들을 얻을 수 있다.

2. Bottom-up 방식으로 small segmented areas들을 합쳐서 더 큰 segmented areas들을 만든다.

3. (2)작업을 반복하여 최종적으로 2000개의 region proposal을 생성한다.

Selective search알고리즘에 의해 2000개의 region proposal이 생성되면 이들을 모두 CNN에 넣기 전에
같은 사이즈로 warp시켜야한다. (CNN output 사이즈를 동일하게 만들기 위해 - For FC layer)

## 2. CNN

<img src='img/Detection/cnn.png' width="500" height="500">

Warp작업을 통해 region proposal 모두 224x224 크기로 되면 CNN 모델에 넣는다.

여기서 CNN은 AlexNet의 거의 구조를 그대로 가져다 썼다.

최종적으로 CNN을 거쳐 각각의 region proposal로부터 4096-dimentional feature vector를 뽑아내고,

이를 통해 고정길이의 Feature Vector를 만들어낸다.

## 3. SVM

<img src='img/Detection/svm.png' width="500" height="500">

CNN모델로부터 feature가 추출되면 Linear SVM을 통해 classification을 진행한다.

위에서 설명했듯이 Classifier로 softmax보다 SVM이 더 좋은 성능을 보였기 때문에 SVM을 채택했다.

SVM은 CNN으로부터 추출된 각각의 feature vector들의 점수를 class별로 매기고, 객체인지 아닌지,
객체라면 어떤 객체인지 등을 판별하는 역할을 하는 Classifier이다.

`+` Selective search로 만든 bounding box는 정확하지 않기 때문에 물체를 정확히 감싸도록 조정해주는 bounding box regression(선형회귀 모델)이 존재한다.

## 단점

1. 여기서 selective search로 2000개의 region proposal을 뽑고 각 영역마다 CNN을 수행하기 때문에

    CNN연산 * 2000 만큼의 시간이 걸려 수행시간이 매우 느리다. 

 

2. CNN, SVM, Bounding Box Regression 총 세가지의 모델이 multi-stage pipelines으로 한 번에 학습되지 않는다.

    각 region proposal 에 대해 ConvNet forward pass를 실행할때 연산을 공유하지 않기에
    end-to-end 로 학습할 수 없다.

    따라서 SVM, bounding box regression에서 학습한 결과가 CNN을 업데이트 시키지 못한다.
    
"그리고 이 두가지 문제를 RoI pooling으로 해결한 Fast R-CNN이 나오게 된다."

---

# 2. Fast R-CNN (2014)
***Fast R-CNN***

원문 :  https://arxiv.org/abs/1504.08083

# 2. Faster R-CNN (2015)
***Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks***

원문 :  https://arxiv.org/abs/1506.01497